In [42]:
# Load libraries
import json
import numpy as np
import pandas as pd
from numpy import log2 as log


In [43]:
with open('feature.txt') as json_file:
    data = json.load(json_file)

In [44]:
#dataframe
df=pd.DataFrame(data)
df=df.transpose() 

In [45]:
def gini(df,features,epsilon):
    target=df.keys()[-1]
    gini = 0
    for i in df[features].unique():
        gini_each_feature = 0
        for j in df[target].unique():
            numerator = len(df[features][df[features]==i][df[target] ==j]) #numerator
            denominator = len(df[features][df[features]==i])  #denominator
            emp_proba = numerator/(denominator+epsilon)    
            # Epsilon is a values near to zero.This is useful to prevent the denominator from being zero
            gini_each_feature += -emp_proba*(1-emp_proba)
            
        fraction2 = denominator/len(df)
        gini += -fraction2*gini_each_feature   #Sums up all the entropy ETaste
        #minimun=np.min(gini)
        
    return(gini)




In [47]:
def min_impurity(df):
    minimum = []
    for key in df.keys()[:-1]:
        minimum.append(gini(df,key,epsilon))
    return df.keys()[:-1][np.argmin(minimum)]

In [48]:
def get_subset(df, node,value):
    return df[df[node] == value].reset_index(drop=True)

In [49]:
def buildTree(df,epsilon,tree=None): 
    Class = df.keys()[-1]   
    
    #Here we build our decision tree

    #Get attribute with minimum values 
    node = min_impurity(df)
    
    #Get distinct value of that attribute
    attValue = np.unique(df[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 

    for value in attValue:
        subset = get_subset(df,node,value)
        
        clValue,counts = np.unique(subset['label'],return_counts=True)                        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subset,epsilon) #Calling the function recursively 
                   
    return tree

In [50]:

epsilon = np.finfo(float).eps
tree=buildTree(df,epsilon)

In [51]:
tree


{'gills': {'no': {'length': {'3': 'yes',
    '4': {'teeth': {'few': 'no', 'many': 'yes'}},
    '5': 'yes'}},
  'yes': 'no'}}